<table class="table table-bordered">
    <tr>
        <th style="width:250px"><img src='https://www.np.edu.sg/images/default-source/default-album/img-logo.png?sfvrsn=764583a6_0' style="width: 100%; height: 125px; "></th>
        <th style="text-align:center;"><h1>Distributed Data Pipelines</h1><h2>Assignment 1 </h2><h3>Diploma in Data Science</h3></th>
    </tr>
</table>

Learning Objectives:
- Design PySpark Based Machine Learning
- Execute PySpark Syntax Correctly
- Evaluate and Select Final Model based on Metrics

You will be **graded on the use of PySpark**, so usage of **Pandas itself should be avoided as much as possible**, especially if a particular native method or function is already available in PySpark. **Penalties will be imposed in such cases.**

## Contents <a id="top"></a>
### [Step 1: Problem Statement Formulation](#step1)
### [Step 2: Exploratory Data Analysis and Data Cleansing](#step2)
   - [2.1. Missing Value Treatment](#step2.1.) 
   - [2.1. Drop Unnecessary Columns](#step2.2.) 
   
   
### [Step 3: Data Wrangling and Transformation](#step3)
   - [3.1. Categorical variable treatment and transformations](#step3.1.) 
   - [3.2. Feature Scaling](#step3.2.)    


### [Step 4: Machine Learning Modelling](#step4)
### [Step 5: Model Evaluation and Selection](#step5)

<hr>

## Report  <a id="report"></a>
### [1. Introduction](#part1.)
   - [1.1. Introduction: Problem Understanding](#part1.1.) 
   - [1.2. Formulate a Value Based Problem Statement](#part1.2.) 

### [2. Exploratory Data Analysis and Data Cleansing](#part2.)
   - [2.1. Missing Value Treatment](#part2.1.) 
      - [2.1.1. Finding missing value](#part2.1.1.)
      - [2.1.2. Dealing with missing value](#part2.1.2.)
      
      
   - [2.2.Drop Unnecessary Columns](#part2.2.)
      - [2.2.1. Lease Commence Date](#part2.2.1.)  
      - [2.2.2. Town](#part2.2.2.)   
     

### [3. Data Wrangling and Transformation](#part3.)
   - [3.1. Categorical Encoding](#part3.1.)
       - [3.1.1. Ordinal Encoding](#part3.1.1.)
       - [3.1.2. One-Hot Encoding](#part3.1.2.)
       
       
   - [3.2. Feature Scaling](#part3.2.)       
       - [3.2.1. No Scaling](#part3.2.1.)       
       - [3.2.2. Standard Scaler](#part3.2.2.)    
       
       
### [4. Machine Learning Modelling](#part4.)       
   - [4.1. Count of Rows and Columns](#part4.1.)       
   - [4.2. Sample of 10 Rows before modelling](#part4.2.)            
   - [4.3. Building Predictive Model](#part4.3.)        
       

### [5. Model Evaluation and Selection ](#part5.)       
   - [5.1. Utilize Model Metrics for Evaluation](#part5.1.)     


### [6. Summary and Further Improvements](#part6.)     
   - [6.1. Summarize findings](#part6.1.)  
   - [6.2. Explain the possible further improvements ](#part6.2.)  

<hr>

In [1]:
# import the packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count,substring # Showing null count
from pyspark.ml.feature import Imputer
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer 
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.regression import LinearRegression

### Step 1: Problem Statement Formulation <a id="step1"></a>
[Back to Top](#top)

In [2]:
spark = SparkSession.builder.appName('Resale Price Prediction Model').getOrCreate()

In [3]:
# load and explore data
# inferSchema auto detects data type
df_pyspark=spark.read.csv('./data/sg_flat_prices_mod.csv', header=True, inferSchema=True) 

In [4]:
# Show first 5 records
df_pyspark.show(5)

+----+-----+----------+---------+-----+-----------------+------------+--------------+--------------+-------------------+---------------+------------+
|year|month|      town|flat_type|block|      street_name|storey_range|floor_area_sqm|    flat_model|lease_commence_date|remaining_lease|resale_price|
+----+-----+----------+---------+-----+-----------------+------------+--------------+--------------+-------------------+---------------+------------+
|2017|    1|ANG MO KIO|   2 ROOM|  406|ANG MO KIO AVE 10|    10 TO 12|          44.0|      Improved|               1979|            736|    232000.0|
|2017|    1|ANG MO KIO|   3 ROOM|  108| ANG MO KIO AVE 4|    01 TO 03|          67.0|New Generation|               1978|            727|    250000.0|
|2017|    1|ANG MO KIO|   3 ROOM|  602| ANG MO KIO AVE 5|    01 TO 03|          67.0|New Generation|               1980|            749|    262000.0|
|2017|    1|ANG MO KIO|   3 ROOM|  465|ANG MO KIO AVE 10|    04 TO 06|          68.0|New Generation|

In [5]:
# Prints data type 
# Strings: town, flat_type, block, street_name, storey_range, flat_model
# Numerical: year, month, floor_area_sqm, lease_commence_date, remaining_lease, resale_price(target)
df_pyspark.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- town: string (nullable = true)
 |-- flat_type: string (nullable = true)
 |-- block: string (nullable = true)
 |-- street_name: string (nullable = true)
 |-- storey_range: string (nullable = true)
 |-- floor_area_sqm: double (nullable = true)
 |-- flat_model: string (nullable = true)
 |-- lease_commence_date: integer (nullable = true)
 |-- remaining_lease: integer (nullable = true)
 |-- resale_price: double (nullable = true)



In [6]:
df_pyspark.columns

['year',
 'month',
 'town',
 'flat_type',
 'block',
 'street_name',
 'storey_range',
 'floor_area_sqm',
 'flat_model',
 'lease_commence_date',
 'remaining_lease',
 'resale_price']

In [7]:
cat_cols = ['town','flat_type','block','street_name','storey_range','flat_model']
num_cols = ['year','month','floor_area_sqm','lease_commence_date','remaining_lease','resale_price']

In [8]:
df_pyspark.select(num_cols).describe().show()
# from here, we can alr see that 'floor_area_sqm' have null values due to count

+-------+------------------+------------------+-----------------+-------------------+------------------+------------------+
|summary|              year|             month|   floor_area_sqm|lease_commence_date|   remaining_lease|      resale_price|
+-------+------------------+------------------+-----------------+-------------------+------------------+------------------+
|  count|             64247|             64247|            64197|              64247|             64247|             64247|
|   mean|2018.0262424704656| 6.779133656046197|97.77009984890256| 1993.6012420813422| 894.6413840334957|438943.70469516085|
| stddev|0.8146939469668695|3.2635673352950514|24.26994610142912| 12.465629502278013|149.62669792791093|153760.65294972394|
|    min|              2017|                 1|             31.0|               1966|               553|          150000.0|
|    max|              2019|                12|            249.0|               2016|              1160|         1205000.0|
+-------

### Step 2: Exploratory Data Analysis and Data Cleansing  <a id="step2"></a>
[Back to Top](#top)

### Missing Value Treatment  <a id="step2.1."></a>
[Back to Top](#top)

In [9]:
# Show null count
# from pyspark.sql.functions import col, isnan, when, count
df_pyspark.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_pyspark.columns]).show()
#There are only null values in the dataframe itself, which is floor_area_sqm with 50 null values

+----+-----+----+---------+-----+-----------+------------+--------------+----------+-------------------+---------------+------------+
|year|month|town|flat_type|block|street_name|storey_range|floor_area_sqm|flat_model|lease_commence_date|remaining_lease|resale_price|
+----+-----+----+---------+-----+-----------+------------+--------------+----------+-------------------+---------------+------------+
|   0|    0|   0|        0|    0|          0|           0|            50|         0|                  0|              0|           0|
+----+-----+----+---------+-----+-----------+------------+--------------+----------+-------------------+---------------+------------+



In [10]:
# Show 5 null records
df_pyspark.filter(df_pyspark.floor_area_sqm.isNull()).show(5)

+----+-----+----------+---------+-----+---------------+------------+--------------+----------+-------------------+---------------+------------+
|year|month|      town|flat_type|block|    street_name|storey_range|floor_area_sqm|flat_model|lease_commence_date|remaining_lease|resale_price|
+----+-----+----------+---------+-----+---------------+------------+--------------+----------+-------------------+---------------+------------+
|2017|    3|QUEENSTOWN|   4 ROOM|  161|    MEI LING ST|    19 TO 21|          null|  Improved|               1970|            628|    556000.0|
|2017|    3| TOA PAYOH|   4 ROOM|  220|LOR 8 TOA PAYOH|    16 TO 18|          null|  Improved|               1976|            698|    383000.0|
|2017|    3| TOA PAYOH|   4 ROOM|  213|LOR 8 TOA PAYOH|    16 TO 18|          null|  Improved|               1975|            683|    390000.0|
|2017|    3| TOA PAYOH|   4 ROOM|  204|  TOA PAYOH NTH|    13 TO 15|          null|  Improved|               1973|            664|    42

In [11]:
# Handling Missing Values by median
imputer = Imputer(
    inputCols=['floor_area_sqm'], 
    outputCols=["{}_imputed".format(c) for c in ['floor_area_sqm']]
    ).setStrategy("median") #  Replace with median

In [12]:
# Add imputation cols to df
df_pyspark_median = imputer.fit(df_pyspark).transform(df_pyspark)
df_pyspark_median.show(5)

+----+-----+----------+---------+-----+-----------------+------------+--------------+--------------+-------------------+---------------+------------+----------------------+
|year|month|      town|flat_type|block|      street_name|storey_range|floor_area_sqm|    flat_model|lease_commence_date|remaining_lease|resale_price|floor_area_sqm_imputed|
+----+-----+----------+---------+-----+-----------------+------------+--------------+--------------+-------------------+---------------+------------+----------------------+
|2017|    1|ANG MO KIO|   2 ROOM|  406|ANG MO KIO AVE 10|    10 TO 12|          44.0|      Improved|               1979|            736|    232000.0|                  44.0|
|2017|    1|ANG MO KIO|   3 ROOM|  108| ANG MO KIO AVE 4|    01 TO 03|          67.0|New Generation|               1978|            727|    250000.0|                  67.0|
|2017|    1|ANG MO KIO|   3 ROOM|  602| ANG MO KIO AVE 5|    01 TO 03|          67.0|New Generation|               1980|            749

In [13]:
# Remove column, not imputed. 'floor_area_sqm_imputed' will be used instead
df_pyspark = df_pyspark_median.drop('floor_area_sqm')

In [14]:
df_pyspark.show(5)

+----+-----+----------+---------+-----+-----------------+------------+--------------+-------------------+---------------+------------+----------------------+
|year|month|      town|flat_type|block|      street_name|storey_range|    flat_model|lease_commence_date|remaining_lease|resale_price|floor_area_sqm_imputed|
+----+-----+----------+---------+-----+-----------------+------------+--------------+-------------------+---------------+------------+----------------------+
|2017|    1|ANG MO KIO|   2 ROOM|  406|ANG MO KIO AVE 10|    10 TO 12|      Improved|               1979|            736|    232000.0|                  44.0|
|2017|    1|ANG MO KIO|   3 ROOM|  108| ANG MO KIO AVE 4|    01 TO 03|New Generation|               1978|            727|    250000.0|                  67.0|
|2017|    1|ANG MO KIO|   3 ROOM|  602| ANG MO KIO AVE 5|    01 TO 03|New Generation|               1980|            749|    262000.0|                  67.0|
|2017|    1|ANG MO KIO|   3 ROOM|  465|ANG MO KIO AV

### Drop Unnecessary Columns  <a id="step2.2."></a>
[Back to Top](#top)

In [15]:
df_pyspark.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- town: string (nullable = true)
 |-- flat_type: string (nullable = true)
 |-- block: string (nullable = true)
 |-- street_name: string (nullable = true)
 |-- storey_range: string (nullable = true)
 |-- flat_model: string (nullable = true)
 |-- lease_commence_date: integer (nullable = true)
 |-- remaining_lease: integer (nullable = true)
 |-- resale_price: double (nullable = true)
 |-- floor_area_sqm_imputed: double (nullable = true)



#### Lease Commence Date

In [16]:
# Since lease_commence_date suggest the date where leasehold tenure starts, in Singapore the lease is 99 years,
# and remaining lease is calculated based on lease_commence_date, hence lease_commence_date is not useful.

# Show top 10 rows with `lease_commence_date` = ‘1970’. 
# We assume that a lease started in Jan 1970, meaning that the lease will end in Jan 2069, 
# (2069 – 2017) * 12 = 52 * 12 = 624 months which matches with some the records shown. 
# `lease_commence_date` and `remaining_lease` provides similar information 
df_pyspark.filter(df_pyspark['lease_commence_date']=="1970").show(10)

+----+-----+---------------+---------+-----+---------------+------------+----------+-------------------+---------------+------------+----------------------+
|year|month|           town|flat_type|block|    street_name|storey_range|flat_model|lease_commence_date|remaining_lease|resale_price|floor_area_sqm_imputed|
+----+-----+---------------+---------+-----+---------------+------------+----------+-------------------+---------------+------------+----------------------+
|2017|    1|    BUKIT MERAH|   3 ROOM|   16|  TAMAN HO SWEE|    04 TO 06|  Standard|               1970|            625|    300000.0|                  54.0|
|2017|    1|    BUKIT MERAH|   4 ROOM|  101| HENDERSON CRES|    01 TO 03|  Standard|               1970|            630|    395000.0|                  74.0|
|2017|    1|KALLANG/WHAMPOA|   2 ROOM|    7|       JLN BATU|    07 TO 09|  Standard|               1970|            629|    276000.0|                  48.0|
|2017|    1|KALLANG/WHAMPOA|   3 ROOM|   27|    JLN BAHAGI

In [17]:
# Drop lease commence date
df_pyspark = df_pyspark.drop('lease_commence_date')
df_pyspark.show(5)

+----+-----+----------+---------+-----+-----------------+------------+--------------+---------------+------------+----------------------+
|year|month|      town|flat_type|block|      street_name|storey_range|    flat_model|remaining_lease|resale_price|floor_area_sqm_imputed|
+----+-----+----------+---------+-----+-----------------+------------+--------------+---------------+------------+----------------------+
|2017|    1|ANG MO KIO|   2 ROOM|  406|ANG MO KIO AVE 10|    10 TO 12|      Improved|            736|    232000.0|                  44.0|
|2017|    1|ANG MO KIO|   3 ROOM|  108| ANG MO KIO AVE 4|    01 TO 03|New Generation|            727|    250000.0|                  67.0|
|2017|    1|ANG MO KIO|   3 ROOM|  602| ANG MO KIO AVE 5|    01 TO 03|New Generation|            749|    262000.0|                  67.0|
|2017|    1|ANG MO KIO|   3 ROOM|  465|ANG MO KIO AVE 10|    04 TO 06|New Generation|            745|    265000.0|                  68.0|
|2017|    1|ANG MO KIO|   3 ROOM| 

In [18]:
df_pyspark.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- town: string (nullable = true)
 |-- flat_type: string (nullable = true)
 |-- block: string (nullable = true)
 |-- street_name: string (nullable = true)
 |-- storey_range: string (nullable = true)
 |-- flat_model: string (nullable = true)
 |-- remaining_lease: integer (nullable = true)
 |-- resale_price: double (nullable = true)
 |-- floor_area_sqm_imputed: double (nullable = true)



### Step 3: Data Wrangling and Transformation  <a id="step3"></a>
[Back to Top](#top)

In [19]:
# consider categorical and numerical variable treatment and transformations

In [20]:
# substring(str, pos, len)
substring_storey = df_pyspark.select(substring('storey_range', 1,2)).collect()

In [21]:
df_pyspark.show(5)

+----+-----+----------+---------+-----+-----------------+------------+--------------+---------------+------------+----------------------+
|year|month|      town|flat_type|block|      street_name|storey_range|    flat_model|remaining_lease|resale_price|floor_area_sqm_imputed|
+----+-----+----------+---------+-----+-----------------+------------+--------------+---------------+------------+----------------------+
|2017|    1|ANG MO KIO|   2 ROOM|  406|ANG MO KIO AVE 10|    10 TO 12|      Improved|            736|    232000.0|                  44.0|
|2017|    1|ANG MO KIO|   3 ROOM|  108| ANG MO KIO AVE 4|    01 TO 03|New Generation|            727|    250000.0|                  67.0|
|2017|    1|ANG MO KIO|   3 ROOM|  602| ANG MO KIO AVE 5|    01 TO 03|New Generation|            749|    262000.0|                  67.0|
|2017|    1|ANG MO KIO|   3 ROOM|  465|ANG MO KIO AVE 10|    04 TO 06|New Generation|            745|    265000.0|                  68.0|
|2017|    1|ANG MO KIO|   3 ROOM| 

### Categorical variable treatment and transformations <a id="step3.1."></a>
[Back to Top](#top)

In [22]:
# Show unique values for 'storey_range'
df_pyspark.select('storey_range').distinct().collect()

[Row(storey_range='43 TO 45'),
 Row(storey_range='37 TO 39'),
 Row(storey_range='10 TO 12'),
 Row(storey_range='04 TO 06'),
 Row(storey_range='40 TO 42'),
 Row(storey_range='16 TO 18'),
 Row(storey_range='19 TO 21'),
 Row(storey_range='34 TO 36'),
 Row(storey_range='31 TO 33'),
 Row(storey_range='13 TO 15'),
 Row(storey_range='46 TO 48'),
 Row(storey_range='22 TO 24'),
 Row(storey_range='25 TO 27'),
 Row(storey_range='07 TO 09'),
 Row(storey_range='01 TO 03'),
 Row(storey_range='28 TO 30'),
 Row(storey_range='49 TO 51')]

In [23]:
# Reduce one hot encoding
mapping= {
        '01 TO 03': '1','04 TO 06': '2','07 TO 09': '3','10 TO 12':'4','13 TO 15' :'5',
        '16 TO 18' :'6','19 TO 21':'7', '22 TO 24':'8', '25 TO 27' : '9', '28 TO 30':'10',
        '31 TO 33':'11', '34 TO 36' : '12', '37 TO 39': '13', '40 TO 42' : '14', '43 TO 45' :'15',
        '46 TO 48' : '16', '49 TO 51' : '17'
    }
# Replace mapping dictionary with column 'storey_range'
df_pyspark = df_pyspark.replace(to_replace=mapping,subset=['storey_range'])
# Show first 5 records after performing mapping
df_pyspark.show(5)

+----+-----+----------+---------+-----+-----------------+------------+--------------+---------------+------------+----------------------+
|year|month|      town|flat_type|block|      street_name|storey_range|    flat_model|remaining_lease|resale_price|floor_area_sqm_imputed|
+----+-----+----------+---------+-----+-----------------+------------+--------------+---------------+------------+----------------------+
|2017|    1|ANG MO KIO|   2 ROOM|  406|ANG MO KIO AVE 10|           4|      Improved|            736|    232000.0|                  44.0|
|2017|    1|ANG MO KIO|   3 ROOM|  108| ANG MO KIO AVE 4|           1|New Generation|            727|    250000.0|                  67.0|
|2017|    1|ANG MO KIO|   3 ROOM|  602| ANG MO KIO AVE 5|           1|New Generation|            749|    262000.0|                  67.0|
|2017|    1|ANG MO KIO|   3 ROOM|  465|ANG MO KIO AVE 10|           2|New Generation|            745|    265000.0|                  68.0|
|2017|    1|ANG MO KIO|   3 ROOM| 

In [24]:
# Convert string column to integer column
df_pyspark = df_pyspark.withColumn("storey",df_pyspark.storey_range.cast('Integer')).drop("storey_range")

In [25]:
# Variables to encode:
strings_used = ["street_name","block",'flat_type','flat_model','town']

stage_string = [StringIndexer(inputCol= c, outputCol= c+"_string_encoded") for c in strings_used]
stage_one_hot = [OneHotEncoder(inputCol= c+"_string_encoded", outputCol= c+ "_one_hot") for c in strings_used]

# Generate pipeline 
ppl = Pipeline(stages= stage_string + stage_one_hot)
pyspark_index_ohe = ppl.fit(df_pyspark).transform(df_pyspark)
pyspark_index_ohe.show(5)

+----+-----+----------+---------+-----+-----------------+--------------+---------------+------------+----------------------+------+--------------------------+--------------------+------------------------+-------------------------+-------------------+-------------------+------------------+-----------------+------------------+--------------+
|year|month|      town|flat_type|block|      street_name|    flat_model|remaining_lease|resale_price|floor_area_sqm_imputed|storey|street_name_string_encoded|block_string_encoded|flat_type_string_encoded|flat_model_string_encoded|town_string_encoded|street_name_one_hot|     block_one_hot|flat_type_one_hot|flat_model_one_hot|  town_one_hot|
+----+-----+----------+---------+-----+-----------------+--------------+---------------+------------+----------------------+------+--------------------------+--------------------+------------------------+-------------------------+-------------------+-------------------+------------------+-----------------+---------

In [26]:
df_pyspark.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- town: string (nullable = true)
 |-- flat_type: string (nullable = true)
 |-- block: string (nullable = true)
 |-- street_name: string (nullable = true)
 |-- flat_model: string (nullable = true)
 |-- remaining_lease: integer (nullable = true)
 |-- resale_price: double (nullable = true)
 |-- floor_area_sqm_imputed: double (nullable = true)
 |-- storey: integer (nullable = true)



In [27]:
pyspark_index_ohe.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- town: string (nullable = true)
 |-- flat_type: string (nullable = true)
 |-- block: string (nullable = true)
 |-- street_name: string (nullable = true)
 |-- flat_model: string (nullable = true)
 |-- remaining_lease: integer (nullable = true)
 |-- resale_price: double (nullable = true)
 |-- floor_area_sqm_imputed: double (nullable = true)
 |-- storey: integer (nullable = true)
 |-- street_name_string_encoded: double (nullable = false)
 |-- block_string_encoded: double (nullable = false)
 |-- flat_type_string_encoded: double (nullable = false)
 |-- flat_model_string_encoded: double (nullable = false)
 |-- town_string_encoded: double (nullable = false)
 |-- street_name_one_hot: vector (nullable = true)
 |-- block_one_hot: vector (nullable = true)
 |-- flat_type_one_hot: vector (nullable = true)
 |-- flat_model_one_hot: vector (nullable = true)
 |-- town_one_hot: vector (nullable = true)



In [28]:
# Remove unwanted columns
pyspark_index_ohe = pyspark_index_ohe.drop("block","street_name",'flat_type','flat_model','town',
                                        'block_string_encoded','street_name_string_encoded','flat_type_string_encoded',
                                          'flat_model_string_encoded','town_string_encoded')

In [29]:
pyspark_index_ohe.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- remaining_lease: integer (nullable = true)
 |-- resale_price: double (nullable = true)
 |-- floor_area_sqm_imputed: double (nullable = true)
 |-- storey: integer (nullable = true)
 |-- street_name_one_hot: vector (nullable = true)
 |-- block_one_hot: vector (nullable = true)
 |-- flat_type_one_hot: vector (nullable = true)
 |-- flat_model_one_hot: vector (nullable = true)
 |-- town_one_hot: vector (nullable = true)



### Train Test Split

In [30]:
(train, test) = pyspark_index_ohe.randomSplit([0.8, 0.2], seed=999)

In [31]:
train.show(10)

+----+-----+---------------+------------+----------------------+------+-------------------+------------------+-----------------+------------------+---------------+
|year|month|remaining_lease|resale_price|floor_area_sqm_imputed|storey|street_name_one_hot|     block_one_hot|flat_type_one_hot|flat_model_one_hot|   town_one_hot|
+----+-----+---------------+------------+----------------------+------+-------------------+------------------+-----------------+------------------+---------------+
|2017|    1|            585|    250000.0|                  67.0|     2|   (540,[49],[1.0])|(2386,[275],[1.0])|    (6,[2],[1.0])|    (18,[1],[1.0])|(25,[19],[1.0])|
|2017|    1|            588|    235000.0|                  60.0|     2|  (540,[284],[1.0])| (2386,[13],[1.0])|    (6,[2],[1.0])|    (18,[7],[1.0])|(25,[18],[1.0])|
|2017|    1|            588|    262000.0|                  60.0|     1|  (540,[284],[1.0])|(2386,[127],[1.0])|    (6,[2],[1.0])|    (18,[7],[1.0])|(25,[18],[1.0])|
|2017|    1|    

In [32]:
test.show(10)

+----+-----+---------------+------------+----------------------+------+-------------------+------------------+-----------------+------------------+---------------+
|year|month|remaining_lease|resale_price|floor_area_sqm_imputed|storey|street_name_one_hot|     block_one_hot|flat_type_one_hot|flat_model_one_hot|   town_one_hot|
+----+-----+---------------+------------+----------------------+------+-------------------+------------------+-----------------+------------------+---------------+
|2017|    1|            588|    230000.0|                  60.0|     1|  (540,[284],[1.0])| (2386,[31],[1.0])|    (6,[2],[1.0])|    (18,[7],[1.0])|(25,[18],[1.0])|
|2017|    1|            588|    334888.0|                  60.0|     6|  (540,[241],[1.0])|(2386,[460],[1.0])|    (6,[2],[1.0])|    (18,[7],[1.0])|(25,[17],[1.0])|
|2017|    1|            589|    340000.0|                  60.0|     5|  (540,[241],[1.0])|(2386,[428],[1.0])|    (6,[2],[1.0])|    (18,[7],[1.0])|(25,[17],[1.0])|
|2017|    1|    

In [33]:
print((train.count(), len(train.columns)))
print((test.count(), len(test.columns)))

(51451, 11)
(12796, 11)


In [34]:
# x_cols only contain features
x_cols = pyspark_index_ohe.columns
x_cols.remove('resale_price')

In [35]:
print(x_cols)

['year', 'month', 'remaining_lease', 'floor_area_sqm_imputed', 'storey', 'street_name_one_hot', 'block_one_hot', 'flat_type_one_hot', 'flat_model_one_hot', 'town_one_hot']


### Feature Scaling <a id="step3.2."></a>
[Back to Top](#top)

### Consolidating X columns

In [36]:
featureassembler=VectorAssembler(inputCols=x_cols,outputCol="Xcols")

In [37]:
train_vectored = featureassembler.transform(train)
# Display 10 records before selecting Xcols and Target Column
train_vectored.show(10)

+----+-----+---------------+------------+----------------------+------+-------------------+------------------+-----------------+------------------+---------------+--------------------+
|year|month|remaining_lease|resale_price|floor_area_sqm_imputed|storey|street_name_one_hot|     block_one_hot|flat_type_one_hot|flat_model_one_hot|   town_one_hot|               Xcols|
+----+-----+---------------+------------+----------------------+------+-------------------+------------------+-----------------+------------------+---------------+--------------------+
|2017|    1|            585|    250000.0|                  67.0|     2|   (540,[49],[1.0])|(2386,[275],[1.0])|    (6,[2],[1.0])|    (18,[1],[1.0])|(25,[19],[1.0])|(2980,[0,1,2,3,4,...|
|2017|    1|            588|    235000.0|                  60.0|     2|  (540,[284],[1.0])| (2386,[13],[1.0])|    (6,[2],[1.0])|    (18,[7],[1.0])|(25,[18],[1.0])|(2980,[0,1,2,3,4,...|
|2017|    1|            588|    262000.0|                  60.0|     1|  (5

In [38]:
test_vectored = featureassembler.transform(test)
# Display 10 records before selecting Xcols and Target Column
test_vectored.show(10)

+----+-----+---------------+------------+----------------------+------+-------------------+------------------+-----------------+------------------+---------------+--------------------+
|year|month|remaining_lease|resale_price|floor_area_sqm_imputed|storey|street_name_one_hot|     block_one_hot|flat_type_one_hot|flat_model_one_hot|   town_one_hot|               Xcols|
+----+-----+---------------+------------+----------------------+------+-------------------+------------------+-----------------+------------------+---------------+--------------------+
|2017|    1|            588|    230000.0|                  60.0|     1|  (540,[284],[1.0])| (2386,[31],[1.0])|    (6,[2],[1.0])|    (18,[7],[1.0])|(25,[18],[1.0])|(2980,[0,1,2,3,4,...|
|2017|    1|            588|    334888.0|                  60.0|     6|  (540,[241],[1.0])|(2386,[460],[1.0])|    (6,[2],[1.0])|    (18,[7],[1.0])|(25,[17],[1.0])|(2980,[0,1,2,3,4,...|
|2017|    1|            589|    340000.0|                  60.0|     5|  (5

### Step 4: Machine Learning Modelling  <a id="step4"></a>
[Back to Top](#top)

In [39]:
# how to we train or test our models?
# use code to show number of rows and columns,
# as well as a sample of 10 rows before heading into Machine Learning Modelling

In [40]:
## Final Tidy-up and Check before Modeling

In [41]:
# Show number of rows and columns before selecting Xcols and Target Column for train data
train_vectored.count(),len(train_vectored.columns)

(51451, 12)

In [42]:
# Show number of rows and columns before selecting Xcols and Target Column for test data
test_vectored.count(),len(test_vectored.columns)

(12796, 12)

In [43]:
# Train data
train_data = train_vectored.select('Xcols',"resale_price")

In [44]:
# Test data
test_data = test_vectored.select("Xcols","resale_price")

In [45]:
# Show rows and columns for train data
train_data.count(),len(train_data.columns)
# only 2 colums, representing X Cols and Target column

(51451, 2)

In [46]:
# Show rows and columns for test data
test_data.count(),len(test_data.columns)

(12796, 2)

In [47]:
train_data.show(10)

+--------------------+------------+
|               Xcols|resale_price|
+--------------------+------------+
|(2980,[0,1,2,3,4,...|    250000.0|
|(2980,[0,1,2,3,4,...|    235000.0|
|(2980,[0,1,2,3,4,...|    262000.0|
|(2980,[0,1,2,3,4,...|    263000.0|
|(2980,[0,1,2,3,4,...|    290000.0|
|(2980,[0,1,2,3,4,...|    307000.0|
|(2980,[0,1,2,3,4,...|    320000.0|
|(2980,[0,1,2,3,4,...|    398000.0|
|(2980,[0,1,2,3,4,...|    228500.0|
|(2980,[0,1,2,3,4,...|    304000.0|
+--------------------+------------+
only showing top 10 rows



In [48]:
test_data.show(10)

+--------------------+------------+
|               Xcols|resale_price|
+--------------------+------------+
|(2980,[0,1,2,3,4,...|    230000.0|
|(2980,[0,1,2,3,4,...|    334888.0|
|(2980,[0,1,2,3,4,...|    340000.0|
|(2980,[0,1,2,3,4,...|    238000.0|
|(2980,[0,1,2,3,4,...|    255000.0|
|(2980,[0,1,2,3,4,...|    283000.0|
|(2980,[0,1,2,3,4,...|    333000.0|
|(2980,[0,1,2,3,4,...|    310000.0|
|(2980,[0,1,2,3,4,...|    285000.0|
|(2980,[0,1,2,3,4,...|    355000.0|
+--------------------+------------+
only showing top 10 rows



In [49]:
# Applying Linear Regression Predictive Algorithm Model
regressor=LinearRegression(featuresCol="Xcols", labelCol='resale_price')
regressor=regressor.fit(train_data)

In [50]:
type(regressor)

pyspark.ml.regression.LinearRegressionModel

In [51]:
# Coefficients -- for equation of linear regression
regressor.coefficients

DenseVector([-1379.7026, 61.9878, 417.0566, 3265.7733, 11761.2783, -15869.8674, -14008.7678, -7777.516, -7428.959, 24901.4243, -158954.912, -72364.3474, 48667.7229, -25457.3031, -61617.4234, -97413.5157, -55332.2049, -83435.1751, -50679.7527, -40070.6672, 2268.1341, 37762.5779, 7897.7572, -82783.2506, 17170.693, 21877.0377, -1513.0295, -42375.7358, -65412.1456, -67189.7084, 11040.9631, 65194.349, -62443.0244, -69588.434, -28090.0203, 37305.3601, 6623.4747, -68.9495, -17476.1311, -44681.7141, -122208.4943, -65675.466, 53241.2724, -6116.7228, -79000.2288, 34591.7648, 67045.4077, 19165.9035, -62523.0842, -51975.3105, -48966.0449, -54190.0366, -96330.4381, 188173.9017, -79548.5923, 20923.1986, 57628.9014, 13275.2333, 110191.6814, -82048.9001, -5058.8281, -92676.1812, 39783.625, -54743.0974, -94357.7499, -21067.3842, 35026.7234, 87425.8692, -19169.9607, 24624.1284, -30009.1066, -67168.5172, -4951.5814, -43162.4025, -59477.5063, 130260.4424, 40861.1235, 30959.1044, 16283.8867, -39133.5334, -

In [52]:
regressor.intercept

2477192.9749766276

In [53]:
# first row values of Xcols_sscaled
train_data.take(1)

[Row(Xcols=SparseVector(2980, {0: 2017.0, 1: 1.0, 2: 585.0, 3: 67.0, 4: 2.0, 54: 1.0, 820: 1.0, 2933: 1.0, 2938: 1.0, 2974: 1.0}), resale_price=250000.0)]

In [54]:
type(train_data)

pyspark.sql.dataframe.DataFrame

### Step 5: Model Evaluation and Selection  <a id="step5"></a>
[Back to Top](#top)

In [55]:
train_pred_results=regressor.evaluate(train_data)

In [56]:
train_pred_results.predictions.show(10)

+--------------------+------------+------------------+
|               Xcols|resale_price|        prediction|
+--------------------+------------+------------------+
|(2980,[0,1,2,3,4,...|    250000.0| 68532.61238711979|
|(2980,[0,1,2,3,4,...|    235000.0|  238245.357754678|
|(2980,[0,1,2,3,4,...|    262000.0|224380.41415909957|
|(2980,[0,1,2,3,4,...|    263000.0| 208972.9527106383|
|(2980,[0,1,2,3,4,...|    290000.0| 272249.4310086649|
|(2980,[0,1,2,3,4,...|    307000.0|305459.51242745435|
|(2980,[0,1,2,3,4,...|    320000.0|323188.73024454014|
|(2980,[0,1,2,3,4,...|    398000.0| 310533.9845438455|
|(2980,[0,1,2,3,4,...|    228500.0|221210.75285224896|
|(2980,[0,1,2,3,4,...|    304000.0|275687.40379474266|
+--------------------+------------+------------------+
only showing top 10 rows



In [57]:
test_pred_results=regressor.evaluate(test_data)

In [58]:
test_pred_results.predictions.show(10)

+--------------------+------------+------------------+
|               Xcols|resale_price|        prediction|
+--------------------+------------+------------------+
|(2980,[0,1,2,3,4,...|    230000.0| 215858.8011313253|
|(2980,[0,1,2,3,4,...|    334888.0|331055.82250227267|
|(2980,[0,1,2,3,4,...|    340000.0|  329198.146037356|
|(2980,[0,1,2,3,4,...|    238000.0|232951.52061653603|
|(2980,[0,1,2,3,4,...|    255000.0| 230446.2640926023|
|(2980,[0,1,2,3,4,...|    283000.0|   276402.73061956|
|(2980,[0,1,2,3,4,...|    333000.0|320610.23351013707|
|(2980,[0,1,2,3,4,...|    310000.0| 302493.1664232188|
|(2980,[0,1,2,3,4,...|    285000.0|  251433.973961771|
|(2980,[0,1,2,3,4,...|    355000.0|318359.64496642863|
+--------------------+------------+------------------+
only showing top 10 rows



In [59]:
# Measurement in MAE, MSE, R squared (R2) and RMSE for train
train_pred_results.meanAbsoluteError, train_pred_results.meanSquaredError, train_pred_results.r2, train_pred_results.rootMeanSquaredError

(26041.56651908974, 1240643015.9536366, 0.947426727337644, 35222.76275299308)

In [60]:
# Measurement in MAE, MSE, R squared (R2) and RMSE for test
test_pred_results.meanAbsoluteError, test_pred_results.meanSquaredError, test_pred_results.r2, test_pred_results.rootMeanSquaredError

(27211.437678631944, 1332623013.2455668, 0.9440476966432306, 36505.10941286941)

<hr>

# Report  

## 1. Introduction <a id="part1."></a>
[Back to Report](#report)

In this Jupyter notebook, we are going to explore the dataset given, “sg_flat_prices_mod.csv", which was collected by the government in the resale market. The dataset includes 12 columns and 64247 rows of data.

## 1.1. Load, Explore Data  and Understand Data <a id="part1.1."></a>


<img src="Pictures/Problem Statement Formation/initSparkSession and Load dataset.png" width="500" Title = "Initializing SparkSession and load dataset">

Firstly, we start by initializing a `SparkSession`. Next, we create a PySpark DataFrame by reading the CSV file provided and printing the first few records of the data frame created.

<img src="Pictures/Problem Statement Formation/first5Records.png" width="700" Title = "Display first 5 records of dataframe">
<img src="Pictures/Problem Statement Formation/printSchema().png" width="700" Title = "Display column name and data type">

Next, to understand the data better, we use `printSchema()` to display the schema of the DataFrame in a tree format with column name and data type.

We will need to understand some variables provided in the dataset before we start exploring our dataset. 
>`lease_commence_date` is the year in which the lease begins.

>`remaining_lease` is the number of months left before the lease ends.

>`resale_price` is the price of the HDB being sold, which is also the target that we are predicting for this model.

## 1.2. Formulate a Value-Based Problem Statement <a id="part1.2."></a>

Following up, we are going to wrangle the data to build a machine-learning model to predict the resale price of any given HDB resale transaction. This model can be used by housing agent companies and regular Singapore residents to help them predict the resale price for HDB flats so that they can have a gauge of the worth of an HDB flat.

## 2. Exploratory Data Analysis and Data Cleansing <a id="part2."></a>
[Back to Report](#report)

## 2.1. Missing Value Treatment <a id="part2.1."></a>
[Back to Report](#report)

### 2.1.1. Finding missing value <a id="part2.1.1."></a>
[Back to Report](#report)

<img src="Pictures/Exploratory Data Analysis and Data Cleansing/missingImputation/FindingNullValues.png" width="700" Title = "Finding null values in dataset">

Missing data is often not permitted in Machine Learning (ML) models hence we will need to either remove or transform it into permitted values to produce a complete dataset that can be trained by ML models. Using, `isnan()` and `isNull()` to find NAN or Null values in the data frame. These data may be missing due to a human error occurring when these transactions are compiled by the government from the resale market. 

### 2.1.2. Dealing with missing value <a id="part2.1.2."></a>
[Back to Report](#report)

<b>One of the techniques is to remove all missing data by Complete Case Analysis (CCA): </b>

<img src="Pictures/Exploratory Data Analysis and Data Cleansing/missingImputation/dropNulls.png" width="350" Title = "Complete Case Analysis Approach">
<img src="Pictures/Exploratory Data Analysis and Data Cleansing/missingImputation/dropNullsMetrics.png" width="600" Title = "Complete Case Analysis Approach">


<b>Mean Imputation: </b>
<img src="Pictures/Exploratory Data Analysis and Data Cleansing/missingImputation/meanImputation.png" width="750" Title = "Mean Imputation">
<img src="Pictures/Exploratory Data Analysis and Data Cleansing/missingImputation/meanImputationMetrics.png" width="600" Title = "Mean Imputation Metrics">

<b>Median Imputation: </b>
<img src="Pictures/Exploratory Data Analysis and Data Cleansing/missingImputation/medianImputation.png" width="750" Title = "Median Imputation">
<img src="Pictures/Exploratory Data Analysis and Data Cleansing/missingImputation/medianImputationMetrics.png" width="600" Title = "Median Imputation Metrics">

From the 3 metrics, we observed that median imputation is best suited for `floor_area_sqm` as it has a relatively lower test MAE and MSE as compared to the other two approaches, suggesting that model can predict the resale price that is closer to the actual value. The test R2 for median imputation is the closest to 1 as compared to the other two, this indicates that median imputation provides a better fit and model performance. Thus, median imputation is preferred.

## 2.2. Drop Unnecessary Columns <a id="part2.2."></a>
[Back to Report](#report)

Some columns may not have much impact in improving the model, hence these columns often need to be removed to help the model learn better and speed  it up, thus making it more efficient.

<b> Metrics Before dropping unnecessary columns </b>

<img src="Pictures/Exploratory Data Analysis and Data Cleansing/dropColumns/beforeDropColumnsMetrics.png" width="650" Title = "Metrics after dropping unnecessary columns">

### 2.2.1. Lease Commence Date <a id="part2.2.1."></a>
[Back to Report](#report)

<b> Removing Lease Commence Date</b>
<img src="Pictures/Exploratory Data Analysis and Data Cleansing/dropColumns/dropLeaseCommenceDate.png" width="700" Title = "Removing Lease Commence Date">
<img src="Pictures/Exploratory Data Analysis and Data Cleansing/dropColumns/dropLeaseCommenceDateMetrics.png" width="650" Title = "Metrics for removing Lease Commence Date">

We observed that removing `lease_commence_date` only resulted in a slight drop in test MAE and MSE. Affecting it to predict a hundred dollars away from the target and made little difference to the R2. As `remaining_lease` provides similar information as `lease_commence_date`, hence we removed `lease_commence_date`.

### 2.2.2. Town <a id="part2.2.2."></a>
[Back to Report](#report)

<b> Removing Town</b>
<img src="Pictures/Exploratory Data Analysis and Data Cleansing/dropColumns/dropTown.png" width="800" Title = "Removing Town">
<img src="Pictures/Exploratory Data Analysis and Data Cleansing/dropColumns/dropTownMetrics.png" width="650" Title = "Metrics for retaining Town">

We observed that removing `lease_commence_date` only resulted in a slight drop in test MAE and MSE. Affecting it to predict a hundred dollars away from the target and made little difference to the R2. As `remaining_lease` provides similar information as `lease_commence_date`, hence we removed `lease_commence_date`.

<b> Metrics after dropping unnecessary columns </b>

<img src="Pictures/Exploratory Data Analysis and Data Cleansing/dropColumns/afterDropColumnsMetrics.png" width="650" Title = "Metrics after dropping unnecessary columns">

## 3. Data Wrangling and Transformation <a id="part3."></a>
[Back to Report](#report)

### 3.1. Categorical Data Encoding <a id="part3.1."></a>
[Back to Report](#report)

### 3.1.1. Ordinal Encoding <a id="part3.1.1."></a>
[Back to Report](#report)

<img src="Pictures/Data Wrangling and Transformation/uniqueValuesStoreyRange.png" width="400" Title = "Show unique values for storey range">
<img src="Pictures/Data Wrangling and Transformation/storeyRangeOrdinal.png" width="650" Title = "Use of Mapping Dictionary to perform Ordinal Encoding">
<img src="Pictures/Data Wrangling and Transformation/storeyRangeOrdinalMetrics.png" width="650" Title = "Metrics for Ordinal Encoding">

Ordinal encoding involves the converting of value in each cell into a number. As values of `storey_range` can be arranged in ascending order, hence, assigning an integer for each string may allow the model to understand and harness the relationship between each string.

### 3.1.2. One-Hot Encoding <a id="part3.1.2."></a>
[Back to Report](#report)

<img src="Pictures/Data Wrangling and Transformation/variablesOHE.png" width="700" Title = "One-Hot Encoding to variables">
<img src="Pictures/Data Wrangling and Transformation/variablesOHEPipeline.png" width="700" Title = "Pipeline for One-Hot Encoding">
<img src="Pictures/Data Wrangling and Transformation/droppingUnwanted.png" width="650" Title = "Drop unwanted columns after OHE">
<img src="Pictures/Data Wrangling and Transformation/variablesOHEMetrics.png" width="650" Title = "Metrics for One-Hot-Encoding">

Categorical data is converted to numerical data using one-hot encoding by splitting a column into multiple columns, which increases memory and is computationally heavy.

### 3.2. Feature Scaling <a id="part3.2."></a>
[Back to Report](#report)

Before scaling happens, we will need to combine the list of columns into a single vector column so that we can train machine learning models like the logistic regression model. The transformer used is VectorAssembler.

<img src="Pictures/Data Wrangling and Transformation/declareXCols.png" width="650" Title = "Declaring X Columns">
<img src="Pictures/Data Wrangling and Transformation/VectorAssembler.png" width="500" Title = "Vector Assembler">

VectorAssembler is a transformer that combines a given list of columns into a single vector column. It is useful for combining raw features and features generated by different feature transformers into a single feature vector, to train ML models like logistic regression and decision trees.

### 3.2.1. No Scaling <a id="part3.2.1."></a>
[Back to Report](#report)

<img src="Pictures/Data Wrangling and Transformation/NoScalingMetrics.png" width="650" Title = "Metrics for no scaling">

### 3.2.2. Standard Scaler <a id="part3.2.2."></a>
[Back to Report](#report)

<img src="Pictures/Data Wrangling and Transformation/StandardScaler.png" width="650" Title = "Standard Scaler">

<img src="Pictures/Data Wrangling and Transformation/ScalingMetrics.png" width="650" Title = "Metrics for Standard Scaler">


Comparing the metrics of train and test data, it was observed that train data was not affected by scaling, whereas test data was much affected by it. The metrics show that test data have better model performance with no scaling applied rather than Standard Scaler applied with a 0.5% increase in R2 when no scaling was applied. Furthermore, the model was able to predict about 900 dollars closer to the target when no scaling was applied. Hence, we decide not to scale the data.

## 4. Machine Learning Modelling<a id="part4."></a>
[Back to Report](#report)

### 4.1. Count of Rows and Columns <a id="part4.1."></a>
[Back to Report](#report)

<img src="Pictures/Machine Learning Modelling/countOfRnCVectored.png" width="650" Title = "Display count of Rows and Columns after VectorAssembler">
<b> Rows for Train and Test Vectored </b>
<img src="Pictures/Machine Learning Modelling/trainVectored.png" width="700" Title = "Display first 10 rows of Train Vectored">
<img src="Pictures/Machine Learning Modelling/testVectored.png" width="700" Title = "Display first 10 rows of Test Vectored">

After combining columns into a single vector column, `train_vectored` and `test_vectored` have 12 columns each with `train_vectored` having 51451 rows and `test_vectored` with 12896 rows. The 12 columns are inclusive of the features after categorical encoding and vector columns. Hence, before loading into the model, we need to define which columns train and test data take in. 

<b> Define Train and Test Data </b>
<img src="Pictures/Machine Learning Modelling/DefineTrainAndTest.png" width="400" Title = "Define Train and Test data">

<img src="Pictures/Machine Learning Modelling/countOfRnCTrainAndTest.png" width="400" Title = "Display count of Rows and Columns for Train and Test data">

After combining columns into a single vector column, `train_vectored` and `test_vectored` have 12 columns each with `train_vectored` having 51451 rows and `test_vectored` with 12896 rows. The 12 columns are inclusive of the features after categorical encoding and vector columns. Hence, before loading into the model, we need to define which columns train and test data take in. 

After defining which are train and test data, there are two columns for train and test data which are X Columns and target column (resale_price), with the same number of rows as train_vectored and test_vectored respectively.

### 4.2. 4.2. Sample of 10 Rows before modelling  <a id="part4.2."></a>
[Back to Report](#report)

<img src="Pictures/Machine Learning Modelling/Sample10RowsTrain.png" width="250" Title = "Display first 10 rows for Train Data">
<img src="Pictures/Machine Learning Modelling/Sample10RowsTest.png" width="250" Title = "Display first 10 rows for Test Data">

From these pictures above, we can see the values of X Columns and `resale_price` (target) which will be loaded into the predictive model later.

### 4.3. Building Predictive Model <a id="part4.3."></a>
[Back to Report](#report)

<img src="Pictures/Machine Learning Modelling/BuildPredictiveModel.png" width="600" Title = "Building Predictive Model">

There are two parameters to input for the Linear Regression model, the features column which is the X Column that we have defined earlier, and the label column which is the target that we want to predict, `resale_price`.

<img src="Pictures/Machine Learning Modelling/coefficients.png" width="700" Title = "Building Predictive Model">

There are a lot of coefficients in the model as we have a lot of columns due to the One-Hot encoding that we applied to categorical data.

### 5. Model Evaluation and Selection  <a id="part5."></a>
[Back to Report](#report)

### 5.1. Utilize Model Metrics for Evaluation  <a id="part5.1."></a>
[Back to Report](#report)

<img src="Pictures/Model Evaluation and Selection/trainPredict.png" width="400" Title = "Prediction for train data">
<img src="Pictures/Model Evaluation and Selection/testPredict.png" width="400" Title = "Prediction for test data">

The pictures above show the top 10 rows of the model’s prediction for train and test data respectively. From the sample, we observed that the model predicts some HDB transactions very closely to the actual resale price and some quite far off from the actual price.

<img src="Pictures/Model Evaluation and Selection/LinearRegressionMetrics.png" width="850" Title = "Prediction for test data">

Hence, to evaluate the performance of the model, we use `MAE (Mean Absolute Error)` which is the mean of absolute error values, and `MSE (Mean Squared Error)` which refers to taking the square of all errors and taking the mean. However, as MSE is trying to let greater error values impact the means square more by exaggerating their importance with square, hence we also use `RMSE (Root Mean Square Error)` which is to take the square root of MSE. In this case, we are still achieving our goal of exaggerating the greater error values but scaling it down for viewers to compare. Lastly, `R-Square (R2)` represents how much the real values vary from the model predictions, measuring in percentages.

From the 4 metrics, we observed that the test MAE is about 27k (k represents thousands), which means that the model can predict +/- 27k away from the resale price for any given HDB resale transaction on average. Whereas MSE and RMSE have a value of 1.3B (B represents billion) and 36k respectively, in which lower MSE and RMSE indicate a better fit. Lastly, R2 has a value of 0.94 which is very close to 1, indicating that model is of good fit.

### 6. Summary and Further Improvements <a id="part6."></a>
[Back to Report](#report)

### 6.1. Summarize findings <a id="part6.1."></a>
[Back to Report](#report)

In summary, we have wrangled the dataset by performing data cleansing and data transformation before loading it into a Linear Regression model, in which we evaluated the model performance based on MAE, MSE, RMSE, and R2. Regarding the MAE, MSE, and RMSE of the model, we can conclude that model prediction will vary from the actual resale price by 27k on average, however, there are cases whereby the model predicts very far away from the actual value. Using R2, we can conclude that the model is a good fit as its value is 0.944, in which a good R2 value ranges from 0.6 to 0.9. Hence, we can conclude that the wrangled data is considered good as it has an R2 score of close to 1 however, it can be further improved.

### 6.2.  Explain the possible further improvements  <a id="part6.2."></a>
[Back to Report](#report)

Possible improvements are to increase the number of records so that the model has more training data to be trained on, which may increase the performance of the model eventually.

Another possible improvement is to perform a numerical transformation to numerical data, as the linear regression model is performed better with normally distributed data, which may increase the performance of the model. 

### "Unlisted" Youtube Link to Video Presentation

In [61]:
# insert your link in this cell, you are allowed to comment it out
# youtube link: 